In [27]:
import numpy as np
import pandas as pd
from gensim.test.utils import datapath
from gensim import utils
import string
import re
import gensim.models
import tempfile
from gensim.models import Word2Vec
import time

# Training our model

In [38]:
# Reading in our sentences from the scrape results
df = pd.read_csv('DATA/scrape_results/public_and_private_sentences.csv')

In [39]:
# Converting to list
corpus = df['sentences'].tolist()

In [40]:
# Break each sentence into list of individual words
sentences = []
for sentence in corpus:
    sentences.append(sentence.split(" "))

In [41]:
# Create custom word2vec model with given parameters (from internet)
custom_model = Word2Vec(window = 10, min_count = 5, workers = 20, negative = 5, sample = 1e-5)

# Build the vocab by giving the sentences
custom_model.build_vocab(sentences, progress_per=10000)

# Train the model on the inputted sentences, epoch = 50, higher epochs give better output but increase time
start = time.time()
custom_model.train(sentences, total_examples=custom_model.corpus_count, epochs=50, report_delay=1)
end = time.time()

print("Time elapsed (seconds): " + str(end - start))

Time elapsed (seconds): 38.530200242996216


In [42]:
# Save the model for re-use
custom_model.save('DATA/Data/custom_model')

In [43]:
# Some small tests
print('Number of words: ' + str(len(custom_model.wv)))

Number of words: 6208


In [44]:
# Test to see what words are similar to pensioen
print("Words most similar to pensioen: ", custom_model.wv.most_similar(positive='pensioen'))

Words most similar to pensioen:  [('keuzepensioen', 0.6008238792419434), ('gestort', 0.5896247029304504), ('ouderdomspensioen', 0.5881916284561157), ('aanvullend', 0.5851923823356628), ('opgebouwd', 0.5826515555381775), ('pensioenleeftijd', 0.5816986560821533), ('opgebouwde', 0.5815556645393372), ('u', 0.5804972052574158), ('aowoverbrugging', 0.575259268283844), ('opbouwen', 0.5752021670341492)]
